In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image


In [2]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            
            iou_score = iou(box1[1:], box2[1:])
            
            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0
    
    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

In [3]:
video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv')
video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)

In [4]:
test_lists = []

video_valid = ['57583_000082', '57586_004152', '57911_000147', '57997_003691', '57680_002206', '58095_004022', '57906_000718', '58005_001254', '57679_003316', '58103_003494', '57998_002181', '58048_000086']
for video_name in video_valid:
    test_lists.append(f'{video_name}_Endzone.mp4')
    test_lists.append(f'{video_name}_Sideline.mp4')

In [5]:
gt_df = video_labels[video_labels['video'].isin(test_lists) & (video_labels['impact']==1) & (video_labels['confidence']>1) & (video_labels['visibility']>0)]

In [6]:
valid_epoch = 27
test_df = pd.read_csv(f'/home/thinh/nfl/mmdet-models/model_a1_44/epoch_{valid_epoch}.csv')
test_df = test_df[test_df['score'] > 0.8]
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score
11,57583,82,Endzone,57583_000082_Endzone.mp4,25,550,18,325,13,0.972518
12,57583,82,Endzone,57583_000082_Endzone.mp4,26,550,19,324,13,0.962200
14,57583,82,Endzone,57583_000082_Endzone.mp4,27,550,19,323,12,0.958351
16,57583,82,Endzone,57583_000082_Endzone.mp4,28,550,19,321,14,0.915843
17,57583,82,Endzone,57583_000082_Endzone.mp4,29,551,19,320,15,0.988935
...,...,...,...,...,...,...,...,...,...,...
7271,58048,86,Sideline,58048_000086_Sideline.mp4,404,540,22,273,25,0.972223
7282,58048,86,Sideline,58048_000086_Sideline.mp4,478,564,19,276,18,0.929445
7284,58048,86,Sideline,58048_000086_Sideline.mp4,479,565,20,274,18,0.877648
7289,58048,86,Sideline,58048_000086_Sideline.mp4,491,555,22,273,25,0.900143


In [7]:
gt_df['bot'] = gt_df['top'] + gt_df['height']
gt_df['right'] = gt_df['left'] + gt_df['width']
test_df['bot'] = test_df['top'] + test_df['height']
test_df['right'] = test_df['left'] + test_df['width']

/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Frame filter

In [8]:
frame_df = pd.read_csv('../frame_impact.csv')
frame_df.head()

,label,frame,video
0,1,64,57787_003413_Endzone.mp4
1,1,193,58106_002918_Endzone.mp4
2,1,217,57913_000218_Sideline.mp4
3,1,25,58098_001193_Sideline.mp4
4,1,143,58005_001612_Endzone.mp4


In [9]:
test_df = pd.merge(test_df, frame_df, on=['video', 'frame']).drop(columns=['label'])
print(test_df.shape)

(2299, 12)


# Recent frame filter

In [10]:
# FILTER
#################
dropIDX = []
for keys in test_df.groupby(['gameKey', 'playID']).size().to_dict().keys():
    tmp_df = test_df.query('gameKey == @keys[0] and playID == @keys[1]')
    
    for index, row in tmp_df.iterrows():
        if row['view'] == 'Endzone':
            check_df = tmp_df.query('view == "Sideline"')
            if check_df['frame'].apply(lambda x: np.abs(x - row['frame']) <= 4).sum() == 0:
                dropIDX.append(index)
        
        if row['view'] == 'Sideline':
            check_df = tmp_df.query('view == "Endzone"')
            if check_df['frame'].apply(lambda x: np.abs(x - row['frame']) <= 4).sum() == 0:
                dropIDX.append(index)

In [11]:
test_df = test_df.drop(index = dropIDX).reset_index(drop = True)
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right
0,57583,82,Endzone,57583_000082_Endzone.mp4,43,565,22,302,16,0.971319,318,587
1,57583,82,Endzone,57583_000082_Endzone.mp4,43,417,22,311,29,0.941931,340,439
2,57583,82,Endzone,57583_000082_Endzone.mp4,44,421,22,311,28,0.991588,339,443
3,57583,82,Endzone,57583_000082_Endzone.mp4,44,441,19,320,15,0.975827,335,460
4,57583,82,Endzone,57583_000082_Endzone.mp4,44,567,21,300,18,0.856364,318,588
...,...,...,...,...,...,...,...,...,...,...,...,...
1245,58048,86,Sideline,58048_000086_Sideline.mp4,159,522,12,342,13,0.874574,355,534
1246,58048,86,Sideline,58048_000086_Sideline.mp4,159,523,12,329,13,0.871513,342,535
1247,58048,86,Sideline,58048_000086_Sideline.mp4,162,505,14,287,14,0.920192,301,519
1248,58048,86,Sideline,58048_000086_Sideline.mp4,169,527,11,278,14,0.849773,292,538


# Duplicate bbox

In [12]:
test_df.head(10)

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right
0,57583,82,Endzone,57583_000082_Endzone.mp4,43,565,22,302,16,0.971319,318,587
1,57583,82,Endzone,57583_000082_Endzone.mp4,43,417,22,311,29,0.941931,340,439
2,57583,82,Endzone,57583_000082_Endzone.mp4,44,421,22,311,28,0.991588,339,443
3,57583,82,Endzone,57583_000082_Endzone.mp4,44,441,19,320,15,0.975827,335,460
4,57583,82,Endzone,57583_000082_Endzone.mp4,44,567,21,300,18,0.856364,318,588
5,57583,82,Endzone,57583_000082_Endzone.mp4,45,424,20,311,24,0.999023,335,444
6,57583,82,Endzone,57583_000082_Endzone.mp4,45,440,17,320,14,0.995095,334,457
7,57583,82,Endzone,57583_000082_Endzone.mp4,45,567,22,297,18,0.842034,315,589
8,57583,82,Endzone,57583_000082_Endzone.mp4,46,426,19,312,23,0.997822,335,445
9,57583,82,Endzone,57583_000082_Endzone.mp4,46,438,18,319,15,0.987991,334,456


In [13]:
# Based on score
# Based on middle
# Based on last

# Check iou

dup_bboxes = []
test_df['bbox_id'] = test_df.apply(lambda x: f'{x.name}_{x.frame}', axis=1)

for video in test_df['video'].unique():
    tmp_df = test_df.query('video == @video')
    match_bbox_dict = {}
    check_ids = set()
    
    for index, row in tmp_df.iterrows():
        frame = row['frame']
        bbox = row[['left', 'top', 'right', 'bot']]
        bbox_id = row['bbox_id']
        if bbox_id in check_ids:
            continue
        
        while True:
            frame += 1
            next_tmp_df = tmp_df[tmp_df['frame'] == frame]
            if next_tmp_df.shape[0] == 0:
                break
            for next_i, next_row in next_tmp_df.iterrows():
                next_bbox = next_row[['left', 'top', 'right', 'bot']]
                next_bbox_id = next_row['bbox_id']
                
                if iou(bbox, next_bbox) > 0.5:
#                     if next_bbox_id in check_ids:
#                         print("ok")
                    check_ids.add(next_bbox_id)
                    if bbox_id not in match_bbox_dict: 
                        match_bbox_dict[bbox_id] = []
                    match_bbox_dict[bbox_id].append(next_bbox_id)
    
    for key, value in match_bbox_dict.items():
        dup_bboxes.extend(value)
#     print(match_bbox_dict)
#     break

In [14]:
# # Based on score
# # Based on middle
# # Based on last

# # Check iou

# keep_bboxes = []
# test_df['bbox_id'] = test_df.apply(lambda x: f'{x.name}_{x.frame}', axis=1)
# score_dict = dict(zip(test_df.bbox_id, test_df.score))

# for video in test_df['video'].unique():
#     tmp_df = test_df.query('video == @video')
#     match_bbox_dict = {}
#     check_ids = set()
    
#     for index, row in tmp_df.iterrows():
#         frame = row['frame']
#         bbox = row[['left', 'top', 'right', 'bot']]
#         bbox_id = row['bbox_id']
#         if bbox_id in check_ids:
#             continue
        
#         while True:
#             frame += 1
#             next_tmp_df = tmp_df[tmp_df['frame'] == frame]
#             if next_tmp_df.shape[0] == 0:
#                 break
#             for next_i, next_row in next_tmp_df.iterrows():
#                 next_bbox = next_row[['left', 'top', 'right', 'bot']]
#                 next_bbox_id = next_row['bbox_id']
                
#                 if iou(bbox, next_bbox) > 0.5:
# #                     if next_bbox_id in check_ids:
# #                         print("ok")
#                     check_ids.add(next_bbox_id)
#                     if bbox_id not in match_bbox_dict: 
#                         match_bbox_dict[bbox_id] = []
#                     match_bbox_dict[bbox_id].append(next_bbox_id)
    
#     for key, value in match_bbox_dict.items():
#         max_bbox_id = key
#         max_score = score_dict[key]
#         for bbox_id in value:
#             bbox_score = score_dict[bbox_id]
#             if bbox_score > max_score:
#                 max_bbox_id = bbox_id
#         keep_bboxes.append(max_bbox_id)
# #     print(match_bbox_dict)
# #     break

In [15]:
# print(len(keep_bboxes))
print(len(set(dup_bboxes)))
print(len(dup_bboxes))

737
767


In [16]:
test_df = test_df[~test_df['bbox_id'].isin(dup_bboxes)].reset_index(drop=True)
# test_df = test_df[test_df['bbox_id'].isin(keep_bboxes)].reset_index(drop=True)

In [17]:
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right,bbox_id
0,57583,82,Endzone,57583_000082_Endzone.mp4,43,565,22,302,16,0.971319,318,587,0_43
1,57583,82,Endzone,57583_000082_Endzone.mp4,43,417,22,311,29,0.941931,340,439,1_43
2,57583,82,Endzone,57583_000082_Endzone.mp4,44,441,19,320,15,0.975827,335,460,3_44
3,57583,82,Endzone,57583_000082_Endzone.mp4,45,424,20,311,24,0.999023,335,444,5_45
4,57583,82,Endzone,57583_000082_Endzone.mp4,46,567,22,294,19,0.909322,313,589,10_46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,58048,86,Sideline,58048_000086_Sideline.mp4,159,522,12,342,13,0.874574,355,534,1245_159
509,58048,86,Sideline,58048_000086_Sideline.mp4,159,523,12,329,13,0.871513,342,535,1246_159
510,58048,86,Sideline,58048_000086_Sideline.mp4,162,505,14,287,14,0.920192,301,519,1247_162
511,58048,86,Sideline,58048_000086_Sideline.mp4,169,527,11,278,14,0.849773,292,538,1248_169


In [18]:
ftp, ffp, ffn = [], [], []
for count, video in enumerate(set(gt_df['video'])):
    pred_boxes = test_df[test_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    gt_boxes = gt_df[gt_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
    ftp.append(tp)
    ffp.append(fp)
    ffn.append(fn)

tp = np.sum(ftp)
fp = np.sum(ffp)
fn = np.sum(ffn)
precision = tp / (tp + fp + 1e-6)
recall =  tp / (tp + fn +1e-6)
f1_score = 2*(precision*recall)/(precision+recall+1e-6)
print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score:.4f}')

TP: 104, FP: 409, FN: 294, PRECISION: 0.2027, RECALL: 0.2613, F1 SCORE: 0.2283


In [19]:
# TP: 256, FP: 9871, FN: 193, PRECISION: 0.0253, RECALL: 0.5702, F1 SCORE: 0.0484
# TP: 35, FP: 276, FN: 414, PRECISION: 0.1125, RECALL: 0.0780, F1 SCORE: 0.0921
# TP: 101, FP: 1317, FN: 348, PRECISION: 0.0712, RECALL: 0.2249, F1 SCORE: 0.1082


# Epoch 4: TP: 24, FP: 115, FN: 425, PRECISION: 0.1727, RECALL: 0.0535, F1 SCORE: 0.0816
# Epoch 5: TP: 97, FP: 898, FN: 352, PRECISION: 0.0975, RECALL: 0.2160, F1 SCORE: 0.1343
# Epoch 6: TP: 80, FP: 472, FN: 369, PRECISION: 0.1449, RECALL: 0.1782, F1 SCORE: 0.1598
# Epoch 7: TP: 35, FP: 138, FN: 414, PRECISION: 0.2023, RECALL: 0.0780, F1 SCORE: 0.1125
# Epoch 8: TP: 34, FP: 156, FN: 415, PRECISION: 0.1789, RECALL: 0.0757, F1 SCORE: 0.1064
# Epoch 30: TP: 85, FP: 600, FN: 364, PRECISION: 0.1241, RECALL: 0.1893, F1 SCORE: 0.1499
# Epoch 15: TP: 93, FP: 617, FN: 356, PRECISION: 0.1310, RECALL: 0.2071, F1 SCORE: 0.1605
# Epoch 20: TP: 94, FP: 663, FN: 355, PRECISION: 0.1242, RECALL: 0.2094, F1 SCORE: 0.1559


# Data 44:
# Epoch 10: TP: 130, FP: 1106, FN: 319, PRECISION: 0.1052, RECALL: 0.2895, F1 SCORE: 0.1543
# Epoch 14: TP: 140, FP: 1108, FN: 309, PRECISION: 0.1122, RECALL: 0.3118, F1 SCORE: 0.1650
# Epoch 14: TP: 121, FP: 408, FN: 277, PRECISION: 0.2287, RECALL: 0.3040, F1 SCORE: 0.2611 (remove duplicate bbox threshold 0.5)
# Epoch 20: TP: 142, FP: 1153, FN: 307, PRECISION: 0.1097, RECALL: 0.3163, F1 SCORE: 0.1628
# Epoch 20: TP: 140, FP: 1038, FN: 309, PRECISION: 0.1188, RECALL: 0.3118, F1 SCORE: 0.1721 (remove duplicate bbox threshold 0.9)
# Epoch 20: TP: 137, FP: 646, FN: 312, PRECISION: 0.1750, RECALL: 0.3051, F1 SCORE: 0.2224 (remove duplicate bbox threshold 0.7)
# Epoch 20: TP: 131, FP: 421, FN: 318, PRECISION: 0.2373, RECALL: 0.2918, F1 SCORE: 0.2617 (remove duplicate bbox threshold 0.5)
# Epoch 20: TP: 123, FP: 429, FN: 275, PRECISION: 0.2228, RECALL: 0.3090, F1 SCORE: 0.2589 (remove duplicate bbox threshold 0.5, correct gt)
# Epoch 20: TP: 104, FP: 250, FN: 345, PRECISION: 0.2938, RECALL: 0.2316, F1 SCORE: 0.2590 (select highest score)


# Albu/remove duplicate bbox threshold 0.5
# Epoch 12: TP: 83, FP: 210, FN: 315, PRECISION: 0.2833, RECALL: 0.2085, F1 SCORE: 0.2402
# Epoch 20: TP: 90, FP: 224, FN: 308, PRECISION: 0.2866, RECALL: 0.2261, F1 SCORE: 0.2528
# Epoch 25: TP: 76, FP: 205, FN: 322, PRECISION: 0.2705, RECALL: 0.1910, F1 SCORE: 0.2239
# Epoch 30: TP: 87, FP: 244, FN: 311, PRECISION: 0.2628, RECALL: 0.2186, F1 SCORE: 0.2387

# A1
# Epoch 20: TP: 105, FP: 407, FN: 293, PRECISION: 0.2051, RECALL: 0.2638, F1 SCORE: 0.2308
# Epoch 25: TP: 101, FP: 386, FN: 297, PRECISION: 0.2074, RECALL: 0.2538, F1 SCORE: 0.2282
# Epoch 25: TP: 127, FP: 539, FN: 271, PRECISION: 0.1907, RECALL: 0.3191, F1 SCORE: 0.2387 (threshold 0.7)
# Epoch 25: TP: 138, FP: 685, FN: 260, PRECISION: 0.1677, RECALL: 0.3467, F1 SCORE: 0.2260 (threshold 0.6)
# Epoch 27: TP: 104, FP: 409, FN: 294, PRECISION: 0.2027, RECALL: 0.2613, F1 SCORE: 0.2283
